In [1]:
setwd('/Users/alexis/IEHS Dropbox/Rager Lab/Alexis_Payton/1_Compartment_Analysis/Expt1.2_Cytokine Distribution Analyses/Input')
Output = ('/Users/alexis/IEHS Dropbox/Rager Lab/Alexis_Payton/1_Compartment_Analysis/Expt1.2_Cytokine Distribution Analyses/Output')
cur_date = '011621'

library(readxl)
library(tidyverse)
library(reshape2)

#reading in file
data_df <- data.frame(read_excel("CytokineData_102920.xlsx", sheet = 2))

#reading in demographics file
demographics_data_df = data.frame(read_excel("SubjectInfo_102920.xlsx", sheet = 2))

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths




In [2]:
#dichotomizing variables
for (i in 1:length(demographics_data_df$Race)){
    if(demographics_data_df$Race[i] != 'W'){
        demographics_data_df$Race[i] = 'NW'
    } 
    if(demographics_data_df$Age[i] <= 30){
        demographics_data_df$Age[i] = 'U' #under 30
    } 
    else {
        demographics_data_df$Age[i] = 'O' #over 30
    }
}

for (i in 1:length(demographics_data_df$Race)){
    if (i != 32){ #ignoring NA value
        if(demographics_data_df$BMI[i] < 25){
            demographics_data_df$BMI[i] = 'N' #normal bmi
    }
        else {
            demographics_data_df$BMI[i] = 'O' #overweight
        }
    }
}

#combining data_df and subject info
data_df = full_join(data_df, demographics_data_df) %>%
    select(!Serum_Cotinine)

Joining, by = c("SubjectNo", "Group", "SubjectID")



In [3]:
head(data_df)

,Original_Identifier,Protein,Conc,Conc_pslog2,Compartment,SubjectNo,Group,SubjectID,Race,Ethnicity,Sex,Age,BMI
,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,E_C_F_002,IFNg,17.642316,4.220509,ELF,1,NS,NS_1,W,NH,F,U,O
2,E_C_F_002,IL10,2.873724,1.953721,ELF,1,NS,NS_1,W,NH,F,U,O
3,E_C_F_002,IL12p70,1.625272,1.392467,ELF,1,NS,NS_1,W,NH,F,U,O
4,E_C_F_002,IL13,36.117692,5.214035,ELF,1,NS,NS_1,W,NH,F,U,O
5,E_C_F_002,IL1b,104.409217,6.719857,ELF,1,NS,NS_1,W,NH,F,U,O
6,E_C_F_002,IL6,21.159536,4.469856,ELF,1,NS,NS_1,W,NH,F,U,O


# Version 1

Comparing cytokine distributions within each dichotomous demographic (stratifying by compartment). For example, how does the distribution of each cytokine differ within CS or Ecig when comparing white people to non-white people across each compartment?

In [5]:
#creating dfs for each stratified demographic variable
white_df = subset(data_df, data_df$Race == 'W')
nonwhite_df = subset(data_df, data_df$Race == 'NW')
male_df = subset(data_df, data_df$Sex == 'M')
female_df = subset(data_df, data_df$Sex == 'F')
under30_df = subset(data_df, data_df$Age == 'U')
over30_df = subset(data_df, data_df$Age == 'O')
normalbmi_df = subset(data_df, data_df$BMI == 'N')
overweight_df = subset(data_df, data_df$BMI == 'O')

#creating vectors to loop through
cytokines = sort(unique(data_df$Protein)[!is.na(unique(data_df$Protein))]) #alphabetizing and removing NAs
compartment = c('NLF','ELF','Sputum','Serum')
smokers = c('CS', 'Ecig')

In [ ]:
#initializing vectors to store values
race_vector = c()
sex_vector = c()
age_vector = c()
bmi_vector = c()

wilcoxon_rank_sum_values_V1 = function(df1, df2, empty_vector){
    #
    for (i in 1:length(compartment)){
       for (j in 1:length(smokers)){
            for(k in 1:length(cytokines)){
                variable1_df = df1 %>%
                    filter(Compartment == compartment[i], Protein == cytokines[k], Group == smokers[j]) %>%
                    select(Compartment, Protein, Conc_pslog2)
                variable2_df = df2 %>%
                    filter(Compartment == compartment[i], Protein == cytokines[k], Group == smokers[j]) %>%
                    select(Compartment, Protein, Conc_pslog2)
                
                #running wilcoxon rank sum
                wilcox_test = wilcox.test(variable1_df$Conc_pslog2, variable2_df$Conc_pslog2)

                #contains smoking status compared, compartment, cytokine, u stat, and p value
                values_vector = c(compartment[i], smokers[j], cytokines[k], wilcox_test$statistic, wilcox_test$p.value)
                empty_vector = c(empty_vector, values_vector)

            }
           
        }
    }
    return(empty_vector)
}

#calling fn
race_wilcoxon_values = wilcoxon_rank_sum_values_V1(white_df, nonwhite_df, race_vector)
sex_wilcoxon_values = wilcoxon_rank_sum_values_V1(male_df, female_df, sex_vector)
age_wilcoxon_values = wilcoxon_rank_sum_values_V1(under30_df, over30_df, age_vector)
bmi_wilcoxon_values = wilcoxon_rank_sum_values_V1(normalbmi_df, overweight_df, bmi_vector)

# Version 2

This time we are comparing cytokine distributions between non-smokers and Ecig or CS within each stratified demographic variable (while continuing to stratify by compartment). For example, how does the distribution of each cytokine differ between NS and CS in subject with normal BMIs across each compartment?

In [6]:
#initializing vectors to store values
white_vector = c()
nonwhite_vector = c()
male_vector = c()
female_vector = c()
under30_vector = c()
over30_vector = c()
normalbmi_vector = c()
overweight_vector = c()

wilcoxon_rank_sum_values_V2 = function(dataframe, empty_vector){
    #"""
    #Running wilcoxon rank sum tests after filtering for compartment, protein, and smoking status using a loop. 
    #Ultimately using this test to compare subjects (NS vs. Ecig or CS) within the same demographic.
    
    #:param: subsetted dataframe, empty vector
    #:output: a 1,056 vector containing the compartment, comparison, protein, u stat, p value

    #"""
    for (i in 1:length(compartment)){
        for (j in 1:length(smokers)){
            for(k in 1:length(cytokines)){
                NS_df = dataframe %>%
                    filter(Compartment == compartment[i], Protein == cytokines[k], Group == 'NS') %>%
                    select(Compartment, Protein, Conc_pslog2)
                smokers_df = dataframe %>%
                    filter(Compartment == compartment[i], Protein == cytokines[k], Group == smokers[j]) %>%
                    select(Compartment, Protein, Conc_pslog2)

                #running wilcoxon rank sum
                wilcox_test = wilcox.test(NS_df$Conc_pslog2, smokers_df$Conc_pslog2)

                #contains smoking status compared, compartment, cytokine, u stat, and p value
                values_vector = c(compartment[i], smokers[j], cytokines[k], wilcox_test$statistic, wilcox_test$p.value)
                empty_vector = c(empty_vector, values_vector)

            }
        }
    }
    
    return(empty_vector)
}

#calling fn
white_wilcoxon_values = wilcoxon_rank_sum_values_V2(white_df, white_vector)
nonwhite_wilcoxon_values = wilcoxon_rank_sum_values_V2(nonwhite_df, nonwhite_vector)
male_wilcoxon_values = wilcoxon_rank_sum_values_V2(male_df, male_vector)
female_wilcoxon_values = wilcoxon_rank_sum_values_V2(female_df, female_vector)
under30_wilcoxon_values = wilcoxon_rank_sum_values_V2(under30_df, under30_vector)
over30_wilcoxon_values = wilcoxon_rank_sum_values_V2(over30_df, over30_vector)
normalbmi_wilcoxon_values = wilcoxon_rank_sum_values_V2(normalbmi_df, normalbmi_vector)
overweight_wilcoxon_values = wilcoxon_rank_sum_values_V2(overweight_df, overweight_vector)

Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning 

Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning 

Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning 

Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning 

Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning 

Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning 

Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning 

Reformatting data from both versions to export

In [13]:
final_table = function(vector, variable){
    #"""
    #Reformatting each vector into matrices to export. 
    
    #:param: vector, demographic variable
    #:output: a 6x176 matrix containing compartment, comparison, protein, u stat, p value, p adj

    #"""
    dim(vector) = c(5, 176)
    table = data.frame(t(vector))
    
    colnames(table) = c('Compartment', 'Comparison', 'Cytokine', 'Stat', 'P Value')
    
    #going back to calculate padj values
    PAdj = c()
    for (i in 1:length(compartment)){
        for(j in 1:length(smokers)){
            single_compartment_df = table %>%
                filter(Compartment == compartment[i], Comparison == smokers[j])
            padj =  p.adjust(single_compartment_df[,5], method = "fdr") 
            PAdj = c(PAdj, padj)
        }
    }
    
    table = cbind(table, PAdj)
    Variable = rep(variable, times = length(table$Compartment))
    table = cbind(Variable, table)
    return(table)
}

#calling fn
# race_table = final_table(race_wilcoxon_values, 'Race')
# sex_table = final_table(sex_wilcoxon_values, 'Sex')
# age_table = final_table(age_wilcoxon_values, 'Age')
# bmi_table = final_table(bmi_wilcoxon_values, 'BMI')

#calling final_table fn
white_table = final_table(white_wilcoxon_values, 'White')
nonwhite_table = final_table(nonwhite_wilcoxon_values, 'Non-White')
male_table = final_table(male_wilcoxon_values, 'Male')
female_table = final_table(female_wilcoxon_values, 'Female')
under30_table = final_table(under30_wilcoxon_values, 'Under 30')
over30_table = final_table(over30_wilcoxon_values, 'Over 30')
normalbmi_table = final_table(normalbmi_wilcoxon_values, 'Normal BMI')
overweight_table = final_table(overweight_wilcoxon_values, 'Overweight')

In [15]:
#creating one final table and exporting
#all_tables_V1 = rbind(race_table, sex_table, age_table, bmi_table)
all_tables_V2 = rbind(white_table, nonwhite_table, male_table, female_table, under30_table, over30_table,
                  normalbmi_table, overweight_table)
#write.csv(all_tables_V1, paste0(Output,"/", cur_date, "_Demographic_Distribution_Analysis_V1.csv"), row.names = FALSE)
write.csv(all_tables_V2, paste0(Output,"/", cur_date, "_Demographic_Distribution_Analysis_V2.csv"), row.names = FALSE)